In [10]:
import pandas as pd
path = "data/sample/test.csv"

df_temp = pd.read_csv(path, header=4)

In [11]:

df_temp = df_temp[["SN"]]
df_temp

,SN
0,S224800005
1,S224800005
2,S224800005
3,S224800005
4,S224800005
5,S224800005


In [12]:
df_temp["Measurement"] = float("NaN")
df_temp

,SN,Measurement
0,S224800005,NaN
1,S224800005,NaN
2,S224800005,NaN
3,S224800005,NaN
4,S224800005,NaN
5,S224800005,NaN


In [13]:
datetime = "23/01/2023 02:57:31 p. m."
date = datetime[:10]
print(date)

23/01/2023


In [15]:
# read the CSV file and store it in a DataFrame
df = pd.read_csv(path, header=4)

# add a new column "Measurement" and initialize it with NaN values
df["Measurement"] = float("NaN")
# loop through each row of the DataFrame
for i, row in df.iterrows():
    if row["Status"] == "Failed":
        measurement = row["FirstFail"]
        # search for the measurement in the header
        measurement_column = df.columns.get_loc(measurement)
        # write the value of the measurement in the new column "Measurement"
        df.at[i, "Measurement"] = df.iloc[i, measurement_column]

# select the desired columns
df = df[["SN", "Date", "CycleTime", "Status", "FirstFail", "Measurement"]]
df['Only_Date'] = df['Date'].str.split(" ").str[0]

df

,SN,Date,CycleTime,Status,FirstFail,Measurement,Only_Date
0,S224800005,23/01/2023 02:57:31 p. m.,198.66s,Failed,UUT Current Measurment,0.183029,23/01/2023
1,S224800005,23/01/2023 03:01:49 p. m.,192.42s,Failed,1.35 VDC Measurment,1.351070,23/01/2023
2,S224800005,23/01/2023 03:05:25 p. m.,192.60s,Passed,NaN,NaN,23/01/2023
3,S224800005,23/01/2023 03:18:01 p. m.,198.97s,Passed,NaN,NaN,23/01/2023
4,S224800005,23/01/2023 03:21:47 p. m.,198.96s,Failed,VDC w/7 VAC Power,1.564014,23/01/2023
5,S224800005,23/01/2023 03:26:07 p. m.,189.78s,Failed,VDC w/18 VAC Power,23.796310,23/01/2023


In [85]:
header_df = {'SN': [], 'Date': [], 'CycleTime': [], 'Status': [], 'FirstFail': [], 'Measurement': []}
main_df = pd.DataFrame(header_df)

# save the new DataFrame to a new CSV file
main_df.to_csv("data/sample/output.csv", index=False)
main_df

,SN,Date,CycleTime,Status,FirstFail,Measurement
